# Curating a Dataset From the ChEMBL Database

In this notebook, we'll download and format data from the [ChEMBL](https://www.ebi.ac.uk/chembl/) database for machine learning (ML) purposes. To begin, we need to identify the **ChEMBL ID** for our target of interest. This is often easiest to do by using the **UniProt ID**, which can typically be found from a [UniProt Website](https://www.uniprot.org/) or looking at a relevant protein structure in the PDB. For this notebook, we will download data for [Epidermal Growth Factor Receptor (EGFR)](https://www.uniprot.org/uniprotkb/P00533/entry), a protein that plays a significant role in cancer.

Our task is made easier by the [chembl-downloader](https://github.com/cthoyt/chembl-downloader) package written Charles Tapley-Hoyt. This library simplifies the process of downloading and querying the ChEMBL database. For more on the chEMBL-downloader please see Charlie's [paper](https://joss.theoj.org/papers/10.21105/joss.08844) in The Journal of Open Source Software.

Install the necessary Python libraries

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  !uv pip install --system pandas chembl_downloader

Import the libraries we will use
- chembl_downloader - allows easy downloads and access to the ChEMBL database
- pandas - enables access to data tables
- sys - provides access to system functionality
- files - allows downloads from Google Colab

In [2]:
import chembl_downloader
import pandas as pd
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
  from google.colab import files

To begin, we will use the chembl-downloader to download a file containing the mapping of UniProt IDs to ChEMBL IDs. Once we've downloaded the file we can look up the ChEMBL ID corresponding to the UniProt ID for EGFR. If you want to look for a different protein, change the `uniprot_id` variable below. Make sure the query below returned 1 row and ChEMBL_ID has a value. 

In [5]:
uniprot_id = 'Q9UNQ0'
uniprot_path = chembl_downloader.download_uniprot_mapping()
uniprot_df = pd.read_csv(uniprot_path,sep="\t",names=["UniProt", "ChEMBL_ID","Name","Target_Type"],comment="#")
result_df = uniprot_df.query("UniProt == @uniprot_id and Target_Type == 'SINGLE PROTEIN'")
chembl_id = None
if len(result_df) > 0:
    chembl_id = result_df.ChEMBL_ID.values[0]
    print(f"ChEMBL_ID = {chembl_id}")
else:
    print(f"No ChEMBL_ID found for {uniprot_id}")
print()
result_df

ConnectionError: HTTPSConnectionPool(host='ftp.ebi.ac.uk', port=443): Max retries exceeded with url: /pub/databases/chembl/ChEMBLdb/latest/README (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x1170eb210>: Failed to resolve 'ftp.ebi.ac.uk' ([Errno 8] nodename nor servname provided, or not known)"))

To access the data corresponding to our target of interest, we need to use a relatively complex SQL query.  I borrowed thiis query from the [OpenADMET](https://github.com/OpenADMET/openadmet-toolkit/blob/main/openadmet/toolkit/database/chembl.py) data curation workflow. The function below encapsulates this query.  We provide the ChEMBL ID as input and the fuction returns the sql query.

**One Important Note About This Query**     
This query returns ChEMBL activity data as pChEMBL, which is the negative log of the molar activity (e.g. pIC50).  The pCHEMBL values are only returned where the **standard_relation** (>,<,=) is "=".  Consequently, this query won't return qualified values like **>30** or **<5**. 

In [1]:
def get_query(chembl_id):
  query =f"""       -- Get all the activity data for a given target using its ChEMBL ID.
        select
            activities.assay_id                  as assay_id,
            activities.doc_id                    as doc_id,
            activities.standard_value            as standard_value,
            molecule_hierarchy.parent_molregno   as molregno,
            compound_structures.canonical_smiles as canonical_smiles,
            compound_structures.standard_inchi_key as standard_inchi_key,
            target_dictionary.tid                as tid,
            target_dictionary.chembl_id          as target_chembl_id,
            pchembl_value                        as pchembl_value,
            molecule_dictionary.pref_name        as compound_name,
            activities.standard_type             as standard_type,
            activities.standard_value            as standard_value,
            activities.standard_relation         as standard_relation,
            activities.standard_units            as standard_units,
            activities.bao_endpoint              as bao_endpoint,
            assays.description                   as assay_description,
            assays.assay_organism                as assay_organism,
            assays.assay_strain                  as assay_strain,
            assays.assay_tissue                  as assay_tissue,
            assays.assay_type                    as assay_type,
            assays.assay_cell_type               as assay_cell_type,
            assays.assay_subcellular_fraction    as assay_subcellular_fraction,
            assays.variant_id                    as variant_id,
            docs.year                            as doc_year,
            docs.journal                         as doc_journal,
            docs.doi                             as doc_doi,
            docs.title                           as doc_title,
            docs.authors                         as doc_authors,
            docs.abstract                        as doc_abstract,
            docs.patent_id                       as doc_patent_id,
            docs.pubmed_id                       as doc_pubmed_id,
            docs.chembl_release_id               as doc_chembl_release_id
        from activities
        join assays ON activities.assay_id = assays.assay_id
        join target_dictionary ON assays.tid = target_dictionary.tid
        join target_components ON target_dictionary.tid = target_components.tid
        join component_class ON target_components.component_id = component_class.component_id
        join docs ON activities.doc_id = docs.doc_id
        join molecule_dictionary ON activities.molregno = molecule_dictionary.molregno
        join molecule_hierarchy ON molecule_dictionary.molregno = molecule_hierarchy.molregno
        join compound_structures ON molecule_hierarchy.parent_molregno = compound_structures.molregno
        where target_chembl_id = '{chembl_id}'
        and activities.data_validity_comment IS null
        and activities.standard_units = 'nM'
        and pchembl_value is not null"""
  return query

Use the function above to get the SQL query.

In [5]:
if chembl_id is not None:
    sql = get_query(chembl_id)
else:
    print("ERROR: chembl_id does not have a value")

Run the SQL query. The ChEMBL downloader will first check to see if you've downloaded the ChEMBL database.  If you haven't, it will download and unpack it for you.  On Google Colab this typically takes 5-10 minutes. Run this cell, then go get a cup of coffee.

Once the database is downloaded, it will run the query and return the result as a Pandas dataframe. We also write the results to a csv file.

In [6]:
%%time
df = chembl_downloader.query(sql)
df.to_csv(f"{chembl_id}.csv", sep='\t', index=False)

CPU times: user 4.98 s, sys: 2.68 s, total: 7.67 s
Wall time: 12.2 s


Let's save the data in case we want to use it later

In [7]:
all_data_output_file = f"{chembl_id}_all_data.csv"
df.to_csv(all_data_output_file)
print(f"wrote all data too {all_data_output_file}")

wrote all data too CHEMBL203_all_data.csv


Let's look at how many of each of the result types are available in the data we downloaded.

In [8]:
df.standard_type.value_counts()

standard_type
IC50       13158
Kd           809
Ki           568
EC50         285
Potency       35
AC50          11
Name: count, dtype: int64

As we can see above we have much more IC50 data than any other datatype.  However, as a recent [paper](https://pubs.acs.org/doi/full/10.1021/acs.jcim.4c00049) from Greg Landrum and Sereina Rinker showed, Ki data tends to be more reliable.  Let's download the Ki data.  We will select the three columns critical for building our ML model.
- canonical_smiles - the SMILES representation for the chemical structure
- molregno - the internal ChEMBL ID for the compounds
- pchembl_value - the negative log of the activity

If you'd prefer to use a different data type like **Kd** or **IC50**, just change the `standard_type` variable below. 


In [9]:
standard_type = "IC50"
output_df = df.query("standard_type == @standard_type")[["canonical_smiles","molregno","pchembl_value"]].groupby(["canonical_smiles","molregno"]).mean()
output_df = output_df.reset_index()
output_df

,canonical_smiles,molregno,pchembl_value
0,Brc1cc2c(NCc3ccccc3)ncnc2s1,1839801,6.620
1,Brc1cc2c(NCc3ccccn3)ncnc2s1,1839818,5.100
2,Brc1cc2c(NCc3cccs3)ncnc2s1,1839821,5.860
3,Brc1cc2c(NCc3ccncc3)ncnc2s1,1839816,5.410
4,Brc1cc2c(Nc3ccccc3)ncnc2s1,1839823,7.100
...,...,...,...
7751,c1coc(-c2nnc(Cc3nc4ccccc4[nH]3)o2)c1,2201823,4.000
7752,c1csc(-c2cc3nccc(Nc4ccc5[nH]ccc5c4)c3s2)c1,126035,9.000
7753,c1csc(-c2cc3ncnc(Nc4ccc5[nH]ccc5c4)c3s2)c1,125696,8.205
7754,c1csc(-c2nnc(Cc3nc4ccccc4[nH]3)o2)c1,2215803,4.000


I prefer to have the SMILES column called "SMILES", let's change that. 

In [10]:
output_df.rename(columns={"canonical_smiles" : "SMILES"},inplace=True)
output_df

,SMILES,molregno,pchembl_value
0,Brc1cc2c(NCc3ccccc3)ncnc2s1,1839801,6.620
1,Brc1cc2c(NCc3ccccn3)ncnc2s1,1839818,5.100
2,Brc1cc2c(NCc3cccs3)ncnc2s1,1839821,5.860
3,Brc1cc2c(NCc3ccncc3)ncnc2s1,1839816,5.410
4,Brc1cc2c(Nc3ccccc3)ncnc2s1,1839823,7.100
...,...,...,...
7751,c1coc(-c2nnc(Cc3nc4ccccc4[nH]3)o2)c1,2201823,4.000
7752,c1csc(-c2cc3nccc(Nc4ccc5[nH]ccc5c4)c3s2)c1,126035,9.000
7753,c1csc(-c2cc3ncnc(Nc4ccc5[nH]ccc5c4)c3s2)c1,125696,8.205
7754,c1csc(-c2nnc(Cc3nc4ccccc4[nH]3)o2)c1,2215803,4.000


Write the data to a csv file.

In [11]:
outfile_name = f"{chembl_id}_{standard_type}.csv"
output_df.to_csv(outfile_name, index=False)
print(f"Saved {outfile_name} with {len(output_df)} rows.")

Saved CHEMBL203_IC50.csv with 7756 rows.


Let's take a quick look at the csv file.

In [12]:
!head {outfile_name}

SMILES,molregno,pchembl_value
Brc1cc2c(NCc3ccccc3)ncnc2s1,1839801,6.62
Brc1cc2c(NCc3ccccn3)ncnc2s1,1839818,5.1
Brc1cc2c(NCc3cccs3)ncnc2s1,1839821,5.86
Brc1cc2c(NCc3ccncc3)ncnc2s1,1839816,5.41
Brc1cc2c(Nc3ccccc3)ncnc2s1,1839823,7.1
Brc1ccc(C2=NN(c3ccccc3)C(c3ccc4ccccc4c3)C2)cc1,1519273,6.15
Brc1ccc(C2=NN(c3ccccc3)C(c3cccc4ccccc34)C2)cc1,1519266,6.31
Brc1ccc(C2=NN(c3nc(-c4ccccc4)cs3)C(c3ccc(Br)cc3)C2)cc1,1171953,5.05
Brc1ccc(CNc2ccc3ncnc(Nc4cccc(Br)c4)c3c2)cc1,1269681,5.5


If you're on Google Colab, you can execute the cell below to download the results to your Downloads directory. If you're running the notebook locally the output file will be in the same directory as this notebook.

In [13]:
if IN_COLAB:
  files.download(outfile_name)